### Настройка окружения

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься долгим деплоем, а сразу начать писать код и работать с airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4

!airflow db init

In [ ]:
# Создадим папку dags
# В этой папке лежат скрипты для создания дагов
# Это стандартное имя для  данной папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

In [ ]:
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.py:68 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.py:68 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.10/dist-packages/flask_appbuilder/models/sqla/interface.

In [ ]:
# Запуск шедулера
!airflow scheduler -D

/usr/local/lib/python3.10/dist-packages/airflow/utils/cli.py:149 SAWarning: relationship 'DagRun.serialized_dag' will copy column serialized_dag.dag_id to column dag_run.dag_id, which conflicts with relationship(s): 'DagRun.task_instances' (copies task_instance.dag_id to dag_run.dag_id), 'TaskInstance.dag_run' (copies task_instance.dag_id to dag_run.dag_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="dag_run,task_instances"' to the 'DagRun.serialized_dag' relationship. (Background on this error at: https://sqlalche.me/e/14/qzyx)
/usr/local/lib/python3.10/dist-packages/airflow/utils/cli.py:149 SAWarning: relationship 'Serialize

In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken  # найти его можно https://dashboard.ngrok.com/get-started/setup 

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/cloud-edge/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http 18273 > /dev/null &

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=3cf3ec19216d8c21cf0630582caa4d1371663013a79354eec6dc5a8c4a2e3c28
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
nohup: redirecting stderr to stdout


### Задание Кастомный Operator

Ваша задача реализовать Operator transfer, который будет перекладывать файл в базу данных sqlite. Для этого у вас есть шаблон кода в котором нужно только реализовать методы по трансферу данных.

Помощь в решении можно найти в данной статье.

```python
from airflow.models.baseoperator import BaseOperator
from airflow.hooks.sqlite_hook import SqliteHook
from airflow.operators.python import PythonOperator
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago 
import pandas as pd

class FileSQLiteTransferHook(SqliteHook):

    def get_pandas_df(self, url_or_path):
      """ Ваш код который читает данные из файла
      """
      pass

    def insert_df_to_db(self, data):
      """ Данный метод вставляет данные в БД
          self.get_conn() это готовый метод SqliteHook для сроздания подключения
      """ 

      data.to_sql('table', con=self.get_conn())

class FileSQLiteTransferOperator(BaseOperator):

    def __init__(self, path, **kwargs):
        super().__init__(**kwargs)
        self.hook = None 
        self.path = path # Путь до файла


    def execute(self, context):
        
        # Создание объекта хука
        self.hook = FileSQLiteTransferHook()

        # Ваш код вызовите метод который 
        # читает данные и затем записывает данные в БД


# Запуск вашего Оператора

dag = DAG('dag', schedule_interval=timedelta(days=1), start_date=days_ago(1))

t1 = FileSQLiteTransferOperator(
  task_id='transfer_data', 
  path='https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/2021-01-04.csv', 
  dag=dag)
```



In [ ]:
## ВАШЕ РЕШЕНИЕ
from airflow.models.baseoperator import BaseOperator
from airflow.hooks.sqlite_hook import SqliteHook
from airflow.operators.python import PythonOperator
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago 
import pandas as pd
import sqlite3
import requests
import io

class FileSQLiteTransferHook(SqliteHook):

    def get_pandas_df(self, url_or_path):
      """ Ваш код который читает данные из файла
      """
      response = requests.get(url_or_path).content
      data = pd.read_csv(io.StringIO(response.decode('utf-8')))
      return data

    def insert_df_to_db(self, data):
      """ Данный метод вставляет данные в БД
          self.get_conn() это готовый метод SqliteHook для сроздания подключения
      """ 

      data.to_sql('table', con=self.get_conn())

class FileSQLiteTransferOperator(BaseOperator):

    def __init__(self, path, **kwargs):
        super().__init__(**kwargs)
        self.hook = None 
        self.path = path # Путь до файла


    def execute(self, context):

        # Создание объекта хука
        self.hook = FileSQLiteTransferHook()
        self.hook.insert_df_to_db(self.hook.get_pandas_df(self.path))

        # Ваш код вызовите метод который 
        # читает данные и затем записывает данные в БД


# Запуск вашего Оператора

dag = DAG('dag', schedule_interval=timedelta(days=1), start_date=days_ago(1))

t1 = FileSQLiteTransferOperator(
  task_id='transfer_data', 
  path='https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data_new/2021-01-04.csv', 
  dag=dag)

<ipython-input-7-c0d22067e679>:3 DeprecationWarning: This module is deprecated. Please use `airflow.providers.sqlite.hooks.sqlite`.

[2023-05-09 18:53:49,293] {utils.py:160} INFO - NumExpr defaulting to 2 threads.


In [ ]:
%load_ext sql

%config SqlMagic.feedback=False

%config SqlMagic.autopandas=True

%sql sqlite:////tmp//sqlite_default.db

%sql select * from 'table'

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * sqlite:////tmp//sqlite_default.db


,index,currency,value,date
0,0,EUR,20,2021-01-04
1,1,EUR,90,2021-01-04
2,2,EUR,38,2021-01-04
3,3,EUR,88,2021-01-04
4,4,EUR,57,2021-01-04
5,5,EUR,39,2021-01-04
6,6,EUR,51,2021-01-04
7,7,EUR,33,2021-01-04
8,8,EUR,68,2021-01-04
